XWing Analysis. 

In [190]:
import random

def throwDices(numberDice, r1threshold, r2threshold):
    r1=r2=r3=0
    for x in range(numberDice):
        randomnumber=random.randint(1,8)
        if(randomnumber<=r1threshold):
            r1+=1
        elif(randomnumber<=r2threshold):
            r2+=1
        else:
            r3+=1
    return(r1,r2,r3)

def attack(attackDice=0, defenseDice=0, focusAttack=0, focusDefense=0, evadeDefense=0, targetLock=0):
    hits=0; nFocusAtt=0; nEvades=0; nFocusDef=0
    (hits, nFocusAtt, blanks)=throwDices(attackDice, 4, 6)
    if(targetLock & ((hits+nFocusAtt*focusAttack)<attackDice)):
        (newHits, newFocusAtt, blanks)=throwDices(attackDice-(hits+nFocusAtt*focusAttack), 4, 6)
        hits+=newHits
        nFocusAtt+=newFocusAtt
    (nEvades, nFocusDef, blanks)=throwDices(defenseDice, 3, 5)
    if(focusAttack):
        hits+=nFocusAtt
    if(focusDefense):
        nEvades+=nFocusDef
        if(nEvades>defenseDice):
            nEvades=defenseDice
    if(evadeDefense):
        if(nEvades<defenseDice):
            nEvades+=1
    hits-=nEvades
    if(hits<0):   
        hits=0
    return hits

In [202]:
def attackStatisticSimulation(attackDice=0, defenseDice=0,focusAttack=0, focusDefense=0, evadeDefense=0, targetLock=0):
    numberAttacks=10000
    result=0
    for x in range(numberAttacks):
        result+=attack(attackDice, defenseDice, focusAttack, focusDefense, evadeDefense, targetLock)/numberAttacks
    return(result)

def printAttackMatrix(focusAttack=0, focusDefense=0, evadeDefense=0, targetLock=0):
    print("\tDefense")
    print("Attack", end = '\t')
    for y in range(5):
            print(y, end = '\t')
    print()
    for x in range(5):
        print(x, end = '\t')
        for y in range(5):
            print(round(attackStatisticSimulation(x,y,focusAttack, focusDefense, evadeDefense, targetLock),2), end = '\t')
        print()


In [203]:
printAttackMatrix(0,0,0,0)

	Defense
Attack	0	1	2	3	4	
0	0.0	0.0	0.0	0.0	0.0	
1	0.5	0.31	0.2	0.12	0.08	
2	1.0	0.7	0.51	0.36	0.25	
3	1.49	1.16	0.9	0.67	0.5	
4	2.01	1.65	1.32	1.07	0.82	


Average damage of standard attacks vs defense dices. No Actions involved

In [207]:
printAttackMatrix(1,1,0,0)

	Defense
Attack	0	1	2	3	4	
0	0.0	0.0	0.0	0.0	0.0	
1	0.75	0.28	0.1	0.04	0.02	
2	1.51	0.92	0.47	0.22	0.11	
3	2.25	1.64	1.08	0.62	0.35	
4	3.0	2.37	1.76	1.21	0.79	


Average damage of standard attacks vs defense dices. Both players focus.

In [205]:
printAttackMatrix(1,0,1,0)

	Defense
Attack	0	1	2	3	4	
0	0.0	0.0	0.0	0.0	0.0	
1	0.75	0.0	0.0	0.0	0.0	
2	1.5	0.57	0.23	0.14	0.08	
3	2.26	1.26	0.75	0.48	0.34	
4	3.0	2.0	1.43	1.04	0.79	


Average damage of standard attacks vs defense dices. Attacker plays the focus, defender evades. In 1-3 defense dice it is better to play the arrow. With 4 dice, the evasion plays like the eye (which is more flexible). The result is the same if defender cannot play evasion.


In [206]:
printAttackMatrix(1,0,0,0)
printAttackMatrix(0,0,0,1)

	Defense
Attack	0	1	2	3	4	
0	0.0	0.0	0.0	0.0	0.0	
1	0.75	0.48	0.3	0.18	0.12	
2	1.5	1.14	0.85	0.62	0.43	
3	2.24	1.89	1.53	1.22	0.93	
4	3.0	2.62	2.28	1.89	1.6	
	Defense
Attack	0	1	2	3	4	
0	0.0	0.0	0.0	0.0	0.0	
1	0.75	0.47	0.29	0.18	0.11	
2	1.51	1.14	0.86	0.61	0.44	
3	2.26	1.87	1.51	1.21	0.94	
4	3.0	2.63	2.26	1.89	1.58	


Impact of target lock. In the first matrix attacker plays "Focus". In the second matrix the Attacker plays Target Lock. Defender does not play the Evasion. 

Incredible: "Focus" is as good as "Target Lock" when attacking always.

In [287]:
def battle(fleetA, fleetB):
    for turns in range(100):
        for initiative in range (10, 0, -1):
            for (ini, atDice, empty, empty) in fleetA:
                # Go from high to low initiatives, and do...
                if(ini==initiative):
                    defDice=fleetB[-1][2]
                    fleetB[-1]=(fleetB[-1][0],fleetB[-1][1],fleetB[-1][2],fleetB[-1][3]-attack(atDice, defDice))
            for (ini, atDice, empty, empty) in fleetB:
                # Go from high to low initiatives, and do...
                if(ini==initiative):
                    defDice=fleetA[-1][2]
                    fleetA[-1]=(fleetA[-1][0],fleetA[-1][1],fleetA[-1][2],fleetA[-1][3]-attack(atDice, defDice))

            #Clean last ship if 0 or less
            if(fleetA[-1][3]<=0):
                del(fleetA[-1])
            if(fleetB[-1][3]<=0):
                del(fleetB[-1])

            #Check for end of battle    
            if(len(fleetA)<1):
                if(len(fleetB)<1):
                    return 0.5
                else:
                    return 0
            if(len(fleetB)<1):
                return 1
    #After 100 turns nobody wins... lets call it a day
    print("TIE")
    return 0.5

def battleStatistics(fleetA, fleetB):
    numberBattles=10000
    result=0
    for x in range(numberBattles):
        result+=battle(fleetA.copy(), fleetB.copy())/numberBattles
    print("Victory A:", round(result,3), "Victory B", round(1-result,3))

In [296]:
#Each ship: (initiative, attack, defense, strength)
battleStatistics([(2,2,3,3),(2,2,3,2)],[(2,3,2,5)])

Victory A: 0.595 Victory B 0.405


In the initial box set, two TIE fighters against a single X Wing, Imperials have a 20% advantage.

In [297]:
#Each ship: (initiative, attack, defense, strength)
battleStatistics([(2,2,3,3),(2,2,3,2)],[(2,3,2,5)])
battleStatistics([(3,2,3,3),(3,2,3,2)],[(2,3,2,5)])
battleStatistics([(1,2,3,3),(1,2,3,2)],[(3,3,2,5)])
battleStatistics([(1,2,3,3),(3,2,3,2)],[(2,3,2,5)])

Victory A: 0.59 Victory B 0.41
Victory A: 0.624 Victory B 0.376
Victory A: 0.499 Victory B 0.501
Victory A: 0.553 Victory B 0.447


Initiative has a certain influence in victory probability, due to the "kill first" factor. Which means that many theoretical encounters finish with both sides being killed at the same time.

In [310]:
battleStatistics([(2,2,3,2)],[(6,3,1,13)])
battleStatistics([(2,2,3,2),(2,2,3,2)],[(6,3,1,13)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(6,3,1,13)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(6,3,1,13)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(6,3,1,13)])

Victory A: 0.001 Victory B 0.999
Victory A: 0.082 Victory B 0.918
Victory A: 0.449 Victory B 0.551
Victory A: 0.848 Victory B 0.152
Victory A: 0.985 Victory B 0.015


Number gives a definitive advantage. Several small ships can beat a big one.

In [311]:
battleStatistics([(2,2,3,2)],[(2,3,2,2)])
battleStatistics([(2,2,3,2),(2,2,3,2)],[(2,3,2,2),(2,3,2,2)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,3,2,2),(2,3,2,2),(2,3,2,2)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,3,2,2),(2,3,2,2),(2,3,2,2),(2,3,2,2)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,3,2,2),(2,3,2,2),(2,3,2,2),(2,3,2,2),(2,3,2,2)])

Victory A: 0.409 Victory B 0.591
Victory A: 0.371 Victory B 0.629
Victory A: 0.346 Victory B 0.654
Victory A: 0.335 Victory B 0.665
Victory A: 0.321 Victory B 0.679


Attack is better than defense. A 3-2 fleet beat 2-3 fleets. The difference grows with fleet strength.

In [309]:
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,2,3,2),(2,2,3,2),(2,2,4,2)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,2,4,2),(2,2,3,2),(2,2,3,2)])
print()
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,2,3,2),(2,2,3,2),(2,3,3,2)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,3,3,2),(2,2,3,2),(2,2,3,2)])
print()
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,2,3,2),(2,2,3,2),(2,3,4,2)])
battleStatistics([(2,2,3,2),(2,2,3,2),(2,2,3,2)],[(2,3,4,2),(2,2,3,2),(2,2,3,2)])

Victory A: 0.396 Victory B 0.604
Victory A: 0.436 Victory B 0.564

Victory A: 0.425 Victory B 0.575
Victory A: 0.278 Victory B 0.722

Victory A: 0.315 Victory B 0.685
Victory A: 0.218 Victory B 0.782


Is it better to go for the strongest first or the weakest first? Go first for the less defended, more damaging ship. If both factors oppose, prime damage: go for the most heavily armed ship.

TO-DO: Text file. Git. Publish.